In [ ]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 


from Artha.nlp_extraction import *
import Artha.data_process as dp
# from Artha.neocoinsdb import Neo
from Artha.neodb import Neo
import Artha.crypto_data as crypto
from tqdm import tqdm
import csv
import json
from collections import Counter
import numpy as np
np.set_printoptions(suppress=True)
import pprint
import math
from datetime import datetime

pp = pprint.PrettyPrinter(indent=1)

# coins database
neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
neo.session.run('''CREATE CONSTRAINT twitter_username IF NOT EXISTS ON (n:Person)
                ASSERT n.username IS UNIQUE''')
neo.session.run('''CREATE CONSTRAINT coin_name IF NOT EXISTS ON (n:Coin)
                ASSERT n.ticker IS UNIQUE''')

In [ ]:
# neo.session.run('''
#                 Merge (n:Person {username: 'checkra_'})
#                 ON CREATE SET n.id ='1356259499431129092',
#                 n.name = 'Checkra',
#                 n.username = 'checkra_'
#         ''')

In [ ]:
print(len(neo.get_nodes()), len(neo.get_relations()))

In [ ]:
neo.clear_db()

In [ ]:
neo.load_coins()
neo.load_follows(dp.load_following("checkra_"))

In [ ]:
path = "../data/follows/"
follows = dp.load_following
for file_loc in os.listdir(path):
    if file_loc != "ucheckra_.csv":
        fs = get_follows(path+file_loc)
        follows.extend(fs)
        # if len(fs) > 2000:
        #     print(file_loc, len(fs))

In [ ]:
%%time
print(len(follows))
neo.load_follows(follows)
neo.load_relations(follows)

In [ ]:
follow_path = "../data/follows/"
tweets_path = "../data/tweets/"

l1, l2 = os.listdir(follow_path), os.listdir(tweets_path)
l1 = [i[1:-4] for i in l1]
l2 = [i[1:-11] for i in l2]
print(len(l1), len(l2))
both = np.intersect1d(l1, l2)
print(len(both))
for person in both[:1]:
    # tweets = dp.load_tweets(person)
    # docs = run_pipeline(person)
    # following = dp.
    for doc in docs:
        if doc._.tickers and "BTC" not in doc._.tickers and "USD" not in doc._.tickers and "ETF" not in doc._.tickers:
            print(doc, doc.ents, doc._.tickers, "\n\n")
            # break
# TODO CLEAN UP ENTITY RECOGNITION

In [ ]:
names = ["BTC_JackSparrow", "razoreth", "Nostranomist", "CryptoKaleo", "nebraskangooner", "Rager", "SavageBTC"]
all_mentions = []

In [ ]:
%%time
for ind, name in enumerate(names):
    # tweets = dp.load_tweets(name)
    docs = run_pipeline(name)
    # print(tweets[0]["created_at"])
    print(ind, name)
    mentions = get_mention_scores(name, docs)
    all_mentions.extend(mentions)

In [ ]:
%%time
neo.load_mentions(all_mentions)

In [ ]:
name = "BTC_JackSparrow"
docs = run_pipeline(name)
    # print(tweets[0]["created_at"])
print(name)
singlementions = get_mention_scores(name, docs)

In [ ]:
for i in singlementions[:10]:
    print(i)

In [ ]:
for doc in docs:
    if "WRX" in doc._.tickers:
        print(doc, doc._.tweeted_at)

In [ ]:
%%time
neo.load_mentions(singlementions)